# 4.19.x Final Assignment

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set(rc={'figure.figsize':(15, 9)})
sns.set(font_scale=1.5) 

### Part 1

The `food_consumption.csv` dataset from the `food` folder contains data on the several countries' food consumption per food category and their respective CO2 emissions. Load it to a DataFrame named `food` and check its contents.

In [ ]:
food = pd.read_csv('data/food_consumption.csv')
food.head()

1. Use the `.describe()` method on the `food` DataFrame to produce descriptive statistics about the `consumption` metric for each class in the `food_category` variable. **Which `food_category` has the highest median value of `food_consumption`?** 

In [ ]:
food.groupby('food_category')['consumption'].describe()

In [ ]:
max_cat_cons = food.groupby('food_category')['consumption'].describe().sort_values('50%', ascending=False).head(1).index[0]
print(f'The food category with the highest median value of consumption is {max_cat_cons}.')

2. In a single chart, plot one boxplot for each `food_category` (11 in total) using the variable `co2_emission` as the metric. **By looking at the chart, which `food_category` has the highest interquartile range (IQR)?**

In [ ]:
sns.boxplot(x='co2_emission', y='food_category', data=food)
plt.show()

> The **beef** `food_category` has clearly the largest IQR (difference between 3rd and 1st quartile). 

3. Looking at the chart from the previous question, which is the `food_category` with the highest median `co2_emission` value? 

> The **beef** `food_category` has clearly the highest median value. 

4. Consider the `consumption` of "poultry" and "fish" across all available `countries`; looking at the table from question 1, the average of poultry consumption (21.22) seems to be higher than that of fish consumption (17.29), but is this difference statistically significant? Create a permutation test in order to assess the null hypothesis that there is no difference between the two means. **Do you accept or reject the null hypothesis?** Explain why. 

In [ ]:
cat1 = 'poultry'
cat2 = 'fish'

food_sub = food[food.food_category==cat1][['country','consumption']].merge(food[food.food_category==cat2][['country','consumption']], on='country', how='inner')
food_sub.columns = ['country', cat1, cat2]
food_sub.head(3)

In [ ]:
cat1_m = food_sub.loc[:,cat1].mean()
cat2_m = food_sub.loc[:,cat2].mean()
print(f'Avg. {cat1} consumption: {round(cat1_m,2)} \nAvg. {cat2} consumption: {round(cat2_m,2)}')

In [ ]:
diff_obs = cat1_m-cat2_m
print(f'Avg. Observed Difference: {round(diff_obs,4)}')

In [ ]:
n = 500
diff_sim = []
for sim in range(n): 
    combined = pd.concat([food_sub.loc[:,cat1], food_sub.loc[:,cat2]])   # combine the two DF columns
    np.random.shuffle(combined.values)   # shuffle vlaues in place
    rndm1 = combined[:int(len(combined)/2)]   # create first half of randomly chosen elements
    rndm2 = combined[int(len(combined)/2):]   # create second half of randomly chosen elements
    diff_m = np.mean(rndm1) - np.mean(rndm2)
    diff_sim.append(diff_m)

In [ ]:
sns.histplot(diff_sim)
plt.axvline(diff_obs, 0, 1, color='r', linestyle='--')
plt.show()

In [ ]:
extreme_obs = sum([el > diff_obs for el in diff_sim])
p_value = sum([el > diff_obs for el in diff_sim])/len(diff_sim)*100
print(f'Nr. of simulated instances more extreme than observed difference: {extreme_obs} out of {len(diff_sim)}')
print(f'p-value = {p_value}')

> The p-value is lower than our chosen significance level of 0.05, which means that there isn't enough evidence to sustain the null hypothesis that there is no difference between the two means. Therefore we **reject the null hypothesis** and conclude that **the difference between the two means is statistically significant**. 

### Part 2

The `world_happiness.csv` dataset from the `data` folder contains a series of variables that can be used as a proxy to a country's evaluation of its own goodness of life. The `happiness_score` metric tries to summarise how "happy" each country is. Load and save the dataset to a DataFrame object named `happy`. As always, familiarise yourself with its contents. 

In [ ]:
happy = pd.read_csv('data/world_happiness.csv')
happy.head()

5. Using a histogram, plot the distribution of the `happiness_score` variable, **which distribution does it resemble?** *(in the answer sheet write the [name of the distribution](https://miro.medium.com/max/962/1*DmPUIjvecL7KllOamoFSDw.png) that best fits the data)* 

In [ ]:
sns.histplot(happy.happiness_score)
plt.show()

> The data looks distributed like a Uniform distribution. We can check it using the [Kolmogorov-Smirnov test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test): 

In [ ]:
from scipy import stats
stat, p_value = stats.kstest(happy.happiness_score, stats.uniform(loc=0.0, scale=160.0).cdf)
print(f'With a p-value={round(p_value,4)}, we accept the null hypothesis that the data is Uniformly distributed. ')

6. Plot a correlation matrix (or a correlation heatmap) between all the numeric variables in the dataset. **Which variable is the least correlated with the `happiness_score` metric?** 

In [ ]:
sns.heatmap(happy.corr(), vmin=-1, vmax=1, annot=True, cmap='BrBG')
plt.show()

In [ ]:
happy.corr()[np.abs(happy.corr()['happiness_score'])==np.min(np.abs(happy.corr()['happiness_score']))]

In [ ]:
min_corr_metric = happy.corr()[np.abs(happy.corr()['happiness_score'])==np.min(np.abs(happy.corr()['happiness_score']))].index[0]
print(f'The {min_corr_metric} metric is the least correlated with the happiness_score variable.')

> The corruption metric is the least correlated with the happiness score variable (remember that a correlation of -0.82 represents a strong inverse relation, weak correlations are represented by values close to zero in absolute terms).

7. You may have noticed that the `corruption` metric has some missing values. **How many countries have a missing corruption value?** After you've answered the question, replace all missing values *in all columns of the DataFrame* with the *respective column's mean value*. 

In [ ]:
happy[happy.corruption.isna()]

In [ ]:
print(f'There are {happy[happy.corruption.isna()].shape[0]} countries with a missing value for the corruption variable. ')

> There are 8 countries with a missing value for the corruption variable. 

In [ ]:
# create list of variables with missing values
cols_with_nan = []
for col in happy.columns: 
    if happy[happy[col].isna()].shape[0] > 0: 
        cols_with_nan.append(col)
cols_with_nan

In [ ]:
# we'll replace each missing value with the respective variable's mean
happy[cols_with_nan].mean()

In [ ]:
# replacing NaNs with column's mean value
happy = happy.fillna(value=happy[cols_with_nan].mean())
# checking that there are no more missing data in the DataFrame
happy[happy.corruption.isna()]

8. Use the `statsmodel` package to create a linear regression model where you use `life_exp` to predict the `happiness_score`. After [refreshing your memory on how to interpret a regression coefficient](https://statisticsbyjim.com/regression/interpret-coefficients-p-values-regression/#:~:text=The%20coefficient%20value%20signifies%20how,in%20isolation%20from%20the%20others.), answer the following question: given the model you just created, **a 1-year increase in life expectancy corresponds to an increase of how many points of the happiness score variable?**. 

In [ ]:
import statsmodels.api as sm
Y = happy['happiness_score']
X = happy[['life_exp']]
X = sm.add_constant(data=X)   # add a constant to the model
model = sm.OLS(endog=Y,exog=X)
results = model.fit()
results.summary()

> Given the model above, an increase of 1-year in life expectancy corresponds to an increase of 5.1 points in the happiness score metric.

9. Split the dataset in `train` and `test` sets, leaving 33% of the data in the latter. Then, using the `sklearn` package, train a linear regression model where you try to predict the `happiness_score` using the following set of predictors: `['social_support', 'freedom', 'generosity', 'life_exp']` *(use a `random_state=42`)*. Calculate the R-squared on the train set and compare it with the R-squared based on the test set. **Would you say that the model is overfitting the training data?** Motivate your answer. 

In [ ]:
y = happy['happiness_score']
X = happy[['social_support', 'freedom', 'generosity', 'life_exp']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
r_squared_train = reg.score(X_train, y_train)
r_squared_test = reg.score(X_test, y_test)
print(f'The R-squared of the model on the training set is {round(r_squared_train,4)}')
print(f'The R-squared of the model on the testing set is {round(r_squared_test,4)}')

> No, the model doesn't seem to be overfitting the data, since its performance on the training and testing sets is very similar. 

10. Using the same model from the previous question, **calculate the MAE on the testing set and report it on the answers Sheet.** Are you satisfyied with the model's performance?

In [ ]:
y_hat = reg.predict(X_test)
mae = np.mean(np.abs(y_test - y_hat))
print(f'The MAE on the test set is: {round(mae,2)}')
mape = np.mean(np.abs((y_test - y_hat)/y_test))*100
print(f'The MAPE on the test set is: {round(mape,2)}%')

> The model's performance could definitely be improved, I wouldn't trust the model to give accurate results since its Mean Absolute Percentage Error is almost 40%. Given the issues of multicollinearity I would probably want to try a different model that doesn't suffer from this problem or I'd try to work on the features, maybe searching for different and more predictive regressors. 

### Bonus

11. <span style="color:red">[BONUS]</span> **Do you notice anything strange when looking at the regression coefficients of the model in the previous answer?** (answer here, not on the response Google Sheet)

In [ ]:
coeff = pd.concat([pd.Series(X_test.columns), pd.Series(reg.coef_)], axis=1)
coeff.columns = ['predictor', 'coefficient']
coeff

> The sign of the first three predictors (`social_support`, `freedom` and `generosity`) are all negative, which means that the more a country is socially supportive, free or generous, the less happy it is which, logically, doesn't make much sense. This should be a fisrt red flag to make you go back to your model and check for statistical significance of the predictors as well as possible traces of multicollinearity. 

12. <span style="color:red">[BONUS]</span> The code in the following cell creates a forecasting model using the `prophet` library. Specifically, it fits an additive model (the effect of the seasonality is added to the trend in order to get forecasts) on a dataframe `df` which contains the number of airline passengers over time. Notice how the seasonality in the forecast is too large at the start of the time series and too small at the end (compared to the data it tries to fit). **Modify the Prophet code to account for the effect of growing seasonality.** 

In [ ]:
# DON NOT DELETE >>> RUN THIS CELL!

from prophet import Prophet

df = pd.read_csv('data/air-passengers.csv')
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(24, freq='MS')
forecast = m.predict(future)
fig = m.plot(forecast)

In [ ]:
m = Prophet(seasonality_mode='multiplicative')
m.fit(df)
future = m.make_future_dataframe(24, freq='MS')
forecast = m.predict(future)
fig = m.plot(forecast)